# Plot time evolution

In this notebook we make time evolution plots at a give radius or radius range of quantities from radial profiles. 


* `ppmstarhub version: 1.2.6` or niagara hub sciconda-astrohub2 kernel
* Parent of this notebook: `ppmstar/projects/H-core-M25/analysis_paper_gas/Hcore-time-evolution-M100+.ipynb` and `Hcore-time-evolution-paperI.ipynb`
* Name of this notebook: `Rprof-time-evolution.ipynb`
* home of this notebook: https://github.com/PPMstar/PPMnotebooks/tree/master/Demos

In [15]:
%pylab ipympl
from astropy import units
from nugridpy import utils as ut
from scipy import interpolate
import sys
# for testing use ppm.py from "demos" branch
sys.path.insert(0,'/Users/fherwig/Repos/PyPPM')

# sys.path.insert(0,'/user/scratch14_ppmstar-lcse/fherwig/PyPPM')
from ppmpy import ppm  
logging.getLogger("matplotlib").setLevel(logging.ERROR)


Populating the interactive namespace from numpy and matplotlib


In [16]:
# Data dictionary to hold the data for the different cases analysed in this nb
data = {}  
runs = []

Create and run below only the cells that contain runs you want to include in the current analysis. Running cells repeatedly will update the data for the corresponding runs

In [17]:
dothis = True
dothis = False
if dothis is True:
    nominal_heat = 0.00003355
    eos = 'g'
    # dir_runs = '/user/niagara_scratch_fherwig/' # execution on ppmstar.org
    dir_runs = '/scratch/f/fherwig/fherwig/'    # execution on niagara jupyterlab
    runs = ['M109','M108','M107','M114','M115',\
              'M106','M105','M116','M110','M117']
    data = ppm.initialize(data,dir, runs, nominal_heat=nominal_heat,\
                      eos=eos)

In [18]:
dothis = True
# dothis = False
if dothis is True:
    nominal_heat = 0.00003355
    eos = 'g'
    dir_runs = '/Users/fherwig/Globus/'    # some data on FH ~/Globus for testing
    runs = ['M239']
    data = ppm.initialize_cases(data, dir_runs, runs, nominal_heat=nominal_heat,\
                      eos=eos)

647 rprof files found in '/Users/fherwig/Globus/M239/prfs/.
Dump numbers range from 0 to 646.


### Plot dump evolution for all runs

In [19]:
all_cases = list(data.keys())
cases = all_cases 

In [20]:

ifig = 113; close(ifig); figure(ifig)
last_dumps = []
for i,case in enumerate(all_cases):
    plot(data[case]['NDump'],data[case]['time(mins)']/60.,ut.linestyle(i)[0],label=case,markevery=200,\
         color=ut.linestylecb(i)[2])
    last_dumps.append(data[case]['NDump'][-1])
legend();xlabel('NDump');ylabel('t / hr')
title('marker every 200 dumps')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'marker every 200 dumps')

### Print run table

In [21]:
ppm.cases_table(data,latex=False)

case                                     X heat   grids max dump max t_hr
M239                                      1000.0  1152      633   2483.6


In [22]:
# Table 1 paper I
ppm.cases_table(data,latex=True)

        ID   & grid  & $t_\mathrm{end}/\hour$ & $\log L/L_\mathrm{nominal}$ \\  
        M239 &  1152 &  2484 & 3.0 \\ 


## RProf GUI
Use RProf GUI to do quick inspection of radial profiles.

In [23]:
all_cases

['M239']

In [24]:
case = all_cases[0]
data[case]['rp'].rprofgui(ifig=int(case[1:3]))

interactive(children=(IntSlider(value=21, description='dump1', max=646), IntSlider(value=635, description='dum…

### Note
The parent notebook demonstrates interpolations used in this notebook. Look there for details, removed here.

## Quantity at pressure-scale height fraction above/below N2-peak

### Time evolution at several radii given in terms of Hp above/below $N^2$ peak

This plot combines different runs, variables and radii. `var_means_dict` allows to save quantities for scaling plots.

In [26]:
runs = ['M107', 'M114', 'M115']
runs = ['M239']

In [28]:
var_means_dict = ppm.time_evol_r_Hp_vars(data, runs, varss  = ['|Ut|'], f_hps = [-1.0,1.0], key = "Demo", fname = '-Ut', logy = False,\
                        ylab="$ U_\mathrm{t} / \mathrm{[km/s]}$",  xlims=(None,None),\
                        ylims=(None,None), legends=0, NDump_range = None, NDump_range_vals = (500,1000),\
                        mrange_interp = (12.,14.,0.0001), sparse = 1,\
                        t_transient_hr = 500, figsizes = (8,5), ifig=1394, warning=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case: M239 f_hp = -1.0
Case: M239 f_hp =  1.0


## Plot the averaged data collected in `var_means_dict`

In [30]:
var_means_dict

{'M239': {-1.0: [9.157766187542423], 1.0: [1.9233988714863617]}}

Plot below var_means. But data went into Table 2, from separate runs and data saved in `var_means_dict`.

In [31]:
data[case].keys()

dict_keys(['path', 'rp', 'rph', 'NDump', 'time(mins)', 'time(secs)', 'rp_one', 'X_Lfactors', 'grids', 'e00os'])

### For given radius plot var for all grids in `runs`

In [42]:
# These are the `f_hp` keys for each run:
for run in runs:
    print(run,list(var_means_dict[run].keys()))

M239 [-1.0, 1.0]


In [52]:
# grids plot assuming there is data available for more than one grid 

f_hp = 1.0  # pick one

grids = []
var_means = []
for run in runs:
    grids.append(data[run]['grids'])
    var_means.append(var_means_dict[run][f_hp])

M239 [1.9233988714863617]


In [58]:
array([(data[run]['grids'],var_means_dict[run][f_hp][0]) for run in runs]).T

array([[1152.        ],
       [   1.92339887]])

FH in progress - try with more data on astrohub

In [ ]:
grids = array(grids)
ifig=1399; close(ifig); figure(ifig,figsize=(6,3))
# plot(log10(grids),log10(var_means),'o-')
plot(log10(grids),var_means,'o-')
xlabel('$\log \mathrm{\ number\ of\ grid\ cells}$')
ylabel('$U$ in var_means')
tight_layout()